In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('2019_nCoV_data.csv')

In [3]:
df['Date'] = df['Date'].apply(pd.to_datetime)
df.drop(['Sno'],axis=1,inplace=True)

In [4]:
df.head()

,Date,Province/State,Country,Last Update,Confirmed,Deaths,Recovered
0,2020-01-22 12:00:00,Anhui,China,01/22/2020 12:00:00,1.0,0.0,0.0
1,2020-01-22 12:00:00,Beijing,China,01/22/2020 12:00:00,14.0,0.0,0.0
2,2020-01-22 12:00:00,Chongqing,China,01/22/2020 12:00:00,6.0,0.0,0.0
3,2020-01-22 12:00:00,Fujian,China,01/22/2020 12:00:00,1.0,0.0,0.0
4,2020-01-22 12:00:00,Gansu,China,01/22/2020 12:00:00,0.0,0.0,0.0


In [5]:
# Countries affected

countries = df['Country'].unique().tolist()
print(countries)

print("\nTotal countries affected by virus: ",len(countries))

['China', 'US', 'Japan', 'Thailand', 'South Korea', 'Mainland China', 'Hong Kong', 'Macau', 'Taiwan', 'Singapore', 'Philippines', 'Malaysia', 'Vietnam', 'Australia', 'Mexico', 'Brazil', 'France', 'Nepal', 'Canada', 'Cambodia', 'Sri Lanka', 'Ivory Coast', 'Germany', 'Finland', 'United Arab Emirates', 'India', 'Italy', 'Sweden', 'Russia', 'Spain', 'UK', 'Belgium']

Total countries affected by virus:  32


In [6]:
#Combining China and Mainland China cases

df['Country'].replace({'Mainland China':'China'},inplace=True)
countries = df['Country'].unique().tolist()
print(countries)
print("\nTotal countries affected by virus: ",len(countries))

['China', 'US', 'Japan', 'Thailand', 'South Korea', 'Hong Kong', 'Macau', 'Taiwan', 'Singapore', 'Philippines', 'Malaysia', 'Vietnam', 'Australia', 'Mexico', 'Brazil', 'France', 'Nepal', 'Canada', 'Cambodia', 'Sri Lanka', 'Ivory Coast', 'Germany', 'Finland', 'United Arab Emirates', 'India', 'Italy', 'Sweden', 'Russia', 'Spain', 'UK', 'Belgium']

Total countries affected by virus:  31


In [7]:
de = df['Date'][-1:].astype('str')
year = int(de.values[0].split('-')[0])
month = int(de.values[0].split('-')[1])
day = int(de.values[0].split('-')[2].split()[0])

from datetime import date
df_latest = df[df['Date'] > pd.Timestamp(date(year,month,day))]
df_latest.head()

,Date,Province/State,Country,Last Update,Confirmed,Deaths,Recovered
700,2020-02-04 22:00:00,Hubei,China,2020-04-02 23:43:00,16678.0,479.0,522.0
701,2020-02-04 22:00:00,Zhejiang,China,2020-05-02 01:33:00,895.0,0.0,63.0
702,2020-02-04 22:00:00,Guangdong,China,2020-05-02 02:03:00,870.0,0.0,32.0
703,2020-02-04 22:00:00,Henan,China,2020-05-02 01:13:00,764.0,2.0,41.0
704,2020-02-04 22:00:00,Hunan,China,2020-05-02 01:43:00,661.0,0.0,35.0


In [8]:
import numpy as np

In [9]:
# Creating a dataframe with total no of confirmed cases for every country
Number_of_countries = len(df_latest['Country'].value_counts())


cases = pd.DataFrame(df_latest.groupby('Country')['Confirmed'].sum())
cases['Country'] = cases.index
cases.index=np.arange(1,Number_of_countries+1)

global_cases = cases[['Country','Confirmed']]
#global_cases.sort_values(by=['Confirmed'],ascending=False)
global_cases

,Country,Confirmed
1,Australia,13.0
2,Belgium,1.0
3,Cambodia,1.0
4,Canada,4.0
5,China,24290.0
6,Finland,1.0
7,France,6.0
8,Germany,12.0
9,Hong Kong,18.0
10,India,3.0


In [10]:
# Importing the world_coordinates dataset
wco = pd.read_csv('world_coordinates.csv')

# Merging the coordinates dataframe with original dataframe
wcd = pd.merge(wco,global_cases,on='Country')
wcd.head()

,Code,Country,latitude,longitude,Confirmed
0,AE,United Arab Emirates,23.424076,53.847818,5.0
1,AU,Australia,-25.274398,133.775136,13.0
2,BE,Belgium,50.503887,4.469936,1.0
3,CA,Canada,56.130366,-106.346771,4.0
4,CN,China,35.861660,104.195397,24290.0


In [11]:
pip install folium

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [12]:
import folium

In [13]:
world_map = folium.Map(location=[30, 40], zoom_start=2.2,tiles='OpenStreetMap')

for lat, lon, value, name in zip(wcd['latitude'], wcd['longitude'], wcd['Confirmed'], wcd['Country']):
    folium.CircleMarker([lat, lon],
                        radius=10,
                        popup = ('<strong>Country</strong>: ' + str(name).capitalize() + '<br>'
                                '<strong>Confirmed Cases</strong>: ' + str(value) + '<br>'),
                        color='red',
                        
                        fill_color='BuPu',
                        fill_opacity=0.3 ).add_to(world_map)
world_map